<a href="https://colab.research.google.com/github/MrCarry123/OrganSegmentation/blob/main/Trainingsegformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install python3.11


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libpython3.11-minimal libpython3.11-stdlib mailcap mime-support
  python3.11-minimal
Suggested packages:
  python3.11-venv binfmt-support
The following NEW packages will be installed:
  libpython3.11-minimal libpython3.11-stdlib mailcap mime-support python3.11
  python3.11-minimal
0 upgraded, 6 newly installed, 0 to remove and 18 not upgraded.
Need to get 5,783 kB of archives.
After this operation, 21.5 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 mailcap all 3.70+nmu1ubuntu1 [23.8 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 mime-support all 3.66 [3,696 B]
Get:3 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy/main amd64 libpython3.11-minimal amd64 3.11.5-1+jammy1 [881 kB]
Get:4 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy/main amd64 python3

In [ ]:
import sys
import os
from google.colab import drive
drive.mount('/content/drive')
sys.path.append('/content/drive/MyDrive/Organsegmentation2/') #change this path

Mounted at /content/drive


In [ ]:
!pip3 install tensorflow==2.13.0
!pip3 install keras==2.13.1
!pip3 install -U segmentation-models

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 5.7 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
import tensorflow.keras

%env SM_FRAMEWORK=tf.keras
import segmentation_models as sm
tf.keras.backend.set_image_data_format('channels_last')

env: SM_FRAMEWORK=tf.keras
Segmentation Models: using `tf.keras` framework.


In [ ]:
import segmentation_models as sm
from glob import glob
import tensorflow as tf
import nibabel as nib
from skimage.transform import resize
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Concatenate, Conv2DTranspose, concatenate, BatchNormalization, Activation, MaxPool2D
import numpy as np
from tqdm import tqdm
from PIL import Image
import random
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam, SGD
# from nilearn.image import resample_img
from tensorflow.keras.utils import to_categorical
import os
import time
from tensorflow.keras.models import Model
from tensorflow.keras.losses import binary_crossentropy

import cv2

In [ ]:
from segformer import SegFormer_B3

In [ ]:
def np_load_and_preprocess(file_path):
    original_image = cv2.imread(str(file_path.decode()))
    # print('original:', original_image.shape)
    # Resize the image to (32, 32) and convert it to (32, 32, 3)
    resized_image = cv2.resize(original_image, (32, 32)).astype(np.float32)
    original_label = cv2.imread(str(file_path.decode()).replace('images', 'labels').replace('volume', 'labels'))

    # Resize the image to (32, 32) and convert it to (32, 32, 3)
    resized_label = cv2.resize(original_label, (32, 32)).astype(np.float32)
    resized_label[resized_label != 0] = 1
    # print('check: ', resized_image.shape, resized_label.shape)
    return resized_image, resized_label

In [ ]:
# Load and preprocess a single file
def load_and_preprocess_file(file_path):
    # Use tf.numpy_function to wrap the function and use NumPy functions inside TensorFlow
    images, labels = tf.numpy_function(np_load_and_preprocess, [file_path], [tf.float32, tf.float32])
    return images, labels

In [ ]:
# change path
# remove [:100] for training with full dataset
volume_files = glob('/content/drive/MyDrive/Organsegmentation2/train_images/*.png')

In [ ]:
len(volume_files)

53760

In [ ]:
train_files, val_files = train_test_split(volume_files, test_size=0.2, random_state=42)

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices(train_files).shuffle(buffer_size=len(train_files))
val_dataset = tf.data.Dataset.from_tensor_slices(val_files)

In [ ]:
# Map the load_and_preprocess_file function to the dataset
train_dataset = train_dataset.map(load_and_preprocess_file, num_parallel_calls=tf.data.AUTOTUNE)
# val_dataset = tf.data.Dataset.from_tensor_slices(val_files)
val_dataset = val_dataset.map(load_and_preprocess_file, num_parallel_calls=tf.data.AUTOTUNE)
# In[14]:

In [ ]:
# Shuffle, batch, cache, and prefetch the dataset
batch_size = 32
train_dataset = train_dataset.shuffle(buffer_size=len(train_files))
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.cache()
train_dataset = train_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
val_dataset = val_dataset.shuffle(buffer_size=len(val_files))
val_dataset = val_dataset.batch(batch_size)
val_dataset = val_dataset.cache()
val_dataset = val_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
opt = Adam(learning_rate=0.001)

In [ ]:
def DiceBCELoss(targets, inputs):
    smooth = 1e-6
    inputs = tf.keras.backend.flatten(inputs)
    targets = tf.keras.backend.flatten(targets)

    BCE = tf.keras.losses.binary_crossentropy(targets, inputs)
    intersection = tf.reduce_sum(targets * inputs)
    dice_loss = 1.0 - (2.0 * intersection + smooth) / (tf.reduce_sum(targets) + tf.reduce_sum(inputs) + smooth)
    Dice_BCE = BCE + dice_loss
    return Dice_BCE

In [ ]:
model = SegFormer_B3(input_shape = (32, 32, 3), num_classes = 3)


In [ ]:
model.summary()
num_layers = len(model.layers)
print(num_layers)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 32, 32, 3)]          0         []                            
                                                                                                  
 mix_vision_transformer (Mi  [(None, 8, 8, 64),           4407212   ['input_1[0][0]']             
 xVisionTransformer)          (None, 4, 4, 128),          8                                       
                              (None, 2, 2, 320),                                                  
                              (None, 1, 1, 512)]                                                  
                                                                                                  
 seg_former_head (SegFormer  (None, 8, 8, 3)              3154179   ['mix_vision_transformer[0

5


In [ ]:
epochs = 5
from tqdm import tqdm
for epoch in range(epochs):
    print("\nStart of epoch %d" % (epoch,))
    start_time = time.time()

    # Training loop
    for step, (x, y) in tqdm(enumerate(train_dataset)):
        # print(x.shape, y.shape)
        with tf.GradientTape(persistent=True) as tape:
            pred = model(x, training=True)
            loss_value = DiceBCELoss(y, pred)

        grads = tape.gradient(loss_value, model.trainable_variables)
        opt.apply_gradients(zip(grads, model.trainable_variables))

    # Validation phase
    val_loss = 0.0
    val_steps = 0
    for step, (val_x, val_y) in tqdm(enumerate(val_dataset)):
        val_pred = model(val_x, training=False)
        val_loss += DiceBCELoss(val_y, val_pred)
        val_steps += 1
    avg_val_loss = val_loss / val_steps

    print("Epoch: %d, Training Loss: %.4f, Validation Loss: %.4f, Time: %.2f seconds" % (epoch, loss_value.numpy(), avg_val_loss, time.time() - start_time))


Start of epoch 0


0it [00:00, ?it/s]WARNING:tensorflow:5 out of the last 5 calls to <function _BaseOptimizer._update_step_xla at 0x7832c8f356c0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
1344it [2:03:13,  5.50s/it]
336it [23:17,  4.16s/it]


Epoch: 0, Training Loss: 1.2937, Validation Loss: 1.2943, Time: 8790.78 seconds

Start of epoch 1


1344it [43:27,  1.94s/it]
336it [03:55,  1.42it/s]


Epoch: 1, Training Loss: 1.2933, Validation Loss: 1.2938, Time: 2843.07 seconds

Start of epoch 2


...

Epoch: 2, Training Loss: 1.2932, Validation Loss: 1.2934, Time: 2813.94 seconds

Start of epoch 3


1344it [43:06,  1.92s/it]
336it [03:55,  1.43it/s]


Epoch: 3, Training Loss: 1.2967, Validation Loss: 1.3207, Time: 2821.44 seconds

Start of epoch 4


1344it [42:43,  1.91s/it]
336it [03:54,  1.43it/s]

Epoch: 4, Training Loss: 1.2932, Validation Loss: 1.2934, Time: 2798.37 seconds


In [ ]:
model.save('sm10.h5')